<a href="https://colab.research.google.com/github/korupolujayanth2004/DeepLearning/blob/main/House__Price_Prediction_using_Multi_layer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#dataset
#https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
import numpy as np

# Step 1: Load training data
train_df = pd.read_csv('train.csv')

# Step 2: Select feature columns and target
features = [
    'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    'TotalBsmtSF', 'GrLivArea', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea'
]
target = 'SalePrice'

# Step 3: Handle missing values in features
X = train_df[features].fillna(train_df[features].mean())
y = train_df[target]

# Step 4: Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Step 6: Build the MLP model with Input Layer
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Step 7: Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Step 8: Predict on validation set
y_val_pred = model.predict(X_val).flatten()

# Step 9: Calculate accuracy metrics on validation set
mae = mean_absolute_error(y_val, y_val_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f'Validation MAE: {mae:.2f}')
print(f'Validation RMSE: {rmse:.2f}')

# Step 10: Show some sample predictions vs actual
print("\nSample predictions on validation data:")
for i in range(5):
    print(f"Actual: {y_val.iloc[i]}, Predicted: {y_val_pred[i]:.2f}")


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - loss: 39191998464.0000 - mae: 182079.9844 - val_loss: 39650557952.0000 - val_mae: 178831.0625
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 38278713344.0000 - mae: 180404.2031 - val_loss: 39616786432.0000 - val_mae: 178750.8750
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 39959748608.0000 - mae: 183397.5781 - val_loss: 39437565952.0000 - val_mae: 178339.8125
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 38213132288.0000 - mae: 180744.3125 - val_loss: 38836162560.0000 - val_mae: 176977.1875
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36247748608.0000 - mae: 176157.6406 - val_loss: 37359063040.0000 - val_mae: 173605.2656
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 35728998400.0000 - mae: 173987.9688 - val_loss: 34400448512.0000 - val_mae: 166676.5000
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 33219911680.0000 - mae: 166770.2500 - val_loss: 2955529830

In [4]:
# --- Function to predict price from new input ---
def predict_price(input_dict):
    input_df = pd.DataFrame([input_dict])
    for feat in features:
        if feat not in input_df.columns:
            input_df[feat] = train_df[feat].mean()
    input_df = input_df[features]
    input_df = input_df.fillna(train_df[features].mean())
    input_scaled = scaler.transform(input_df)
    pred_array = model.predict(input_scaled)
    pred_price = pred_array[0, 0]  # Extract scalar properly
    return pred_price

# Example usage of prediction function
example_input = {
    'LotArea': 8450,
    'OverallQual': 7,
    'OverallCond': 5,
    'YearBuilt': 2003,
    'YearRemodAdd': 2003,
    'TotalBsmtSF': 856,
    'GrLivArea': 1710,
    'FullBath': 2,
    'HalfBath': 1,
    'BedroomAbvGr': 3,
    'TotRmsAbvGrd': 8,
    'GarageCars': 2,
    'GarageArea': 548
}

predicted_price = predict_price(example_input)
print(f"\nPredicted house price for example input: ${predicted_price:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step

Predicted house price for example input: $208654.31
